In [1]:
import json
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment, Experiment
from azureml.data import FileDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.core.compute import ComputeTarget, AmlCompute


print(azureml.core.VERSION)
version = dict(zip(['major','minor','patch'], azureml.core.VERSION.split('.')))
ws = Workspace.from_config()

1.10.0


# metadata dataset 


metdata dataset will be used to pass metadata information about the images as a side input to parallel run step. this file need to be mounted on every node of the cluster and hence why side input is being used. 

the data set will point to the root of the metadata folder

file name is passed in as an argument to the python parallel run step

python entry script can then point to the file on the mounted folder and load the metadata as a pandas dataframe

since the path to metadata will change with each run, it will be passed in as a `FileDataset`


In [3]:
datastore_name = 'godzilla'
if datastore_name in ws.datastores:
    datastore = ws.datastores[datastore_name]
    if datastore and type(datastore) is Datastore: 
        print('Found datastore: ' + datastore_name)
else: 
    print('datastore not found...')

images_dataset_name = 'images_partition'
path_on_datastore = datastore.path('images')
input_images_dataset = Dataset.File.from_files(path=path_on_datastore, validate=False)

In [8]:
metadata_ds = Dataset.File.from_files(path=[(datastore, 'metadata')])
metadata_ds.register(workspace=ws, name='metadata_ds', description='anpr input images metadata')

{
  "source": [
    "('godzilla', 'metadata')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "c2a04b5d-a5e1-4adb-92ad-5055edce627c",
    "name": "metadata_ds",
    "version": 1,
    "description": "anpr input images metadata",
    "workspace": "Workspace.create(name='magaml', subscription_id='907c8efc-c2c8-4c49-a4e1-aeb880e10c88', resource_group='aml')"
  }
}

# check data set 

In [9]:
metadata_ds.to_path()

['/20200810_images.csv', '/images_20200810.csv']

In [11]:
ds1 = Dataset.get_by_name(workspace=ws, name='metadata_ds')
ds1.to_path()

['/20200810_images.csv', '/images_20200810.csv']